In [1]:
import QpixAsicArray as qparray
from QpixAsic import PixelHit, ProcQueue
import random
import numpy as np
random.seed(2)
qpa = qparray.QpixAsicArray(4, 4, debug=0.0)

In [2]:
# timeNow = 1.
# timeEnd = 4.0
# tickNow = 50e6
# deltaT = 1.0
# deltaTick = 50e6
# timeEpsilon = 1e-6
# eventTimes = []
# hitsPerEvent, eventTimes = [], []
# procQueue = ProcQueue()
    
# while(timeNow < timeEnd):

#     items = 0
#     topProc = []
#     midProc = []
#     botProc = []
#     lenRecv = []
#     asicRow = []
#     asicCol = []

#     for asic in qpa:
#         newProcessItems = asic.Process(timeNow - timeEpsilon)
#         if newProcessItems:
#             print("WARNING: ASIC had things left to do at next maor time step")

#     procQueue.AddQueueItem(qpa[0][0], 3, PixelHit(tickNow, [], None, None), timeNow)

#     while(procQueue.Length() > 0):

#         nextItem = procQueue.PopQueue()
#         items += 1

#         topProc.append(qpa.ProcessArray(procQueue, nextItem.inTime))

#         newProcessItems = nextItem.asic.ReceiveData(nextItem)
#         if newProcessItems:
#             midProc.append(1)
#             lenRecv.append(len(newProcessItems))
#             for item in newProcessItems:
#                 procQueue.AddQueueItem(*item)
#         else:
#             midProc.append(0)
#             lenRecv.append(0)

#         botProc.append(qpa.ProcessArray(procQueue, nextItem.inTime))

#     eventTimes.append(qpa._daqNode._absTimeNow-timeNow)
#     hitsPerEvent.append(qpa._daqNode.daqHits)
#     qpa._daqNode.daqHits = 0
#     timeNow += deltaT
#     tickNow += deltaTick
#     # print("ran through this many items:", items)
#     # print('topProc:', np.sum(topProc), len(topProc), topProc[:20])
#     # print('midProc:  ', np.sum(midProc), len(midProc), midProc[:20])
#     # print('botProc:  ', np.sum(botProc), len(botProc), botProc[:20])
#     # print('lenRecv:  ', np.sum(lenRecv), len(lenRecv), lenRecv[:20])
#     # print('sum:', items-np.sum(topProc+midProc+botProc))

In [3]:
qpa._daqNode._debugLevel = 0
qpa.Calibrate(1)
qpa.Calibrate(1)
print(qpa._daqNode.daqHits)

performing calibration..
received data! 265
received data! 493
received data! 488
received data! 795
received data! 811
received data! 753
received data! 974
received data! 993
received data! 1333
received data! 1014
received data! 970
received data! 1310
received data! 1373
received data! 1563
received data! 1546
received data! 1840
calibration complete in 98 steps!
performing calibration..
received data! 55845682
received data! 48343359
received data! 47313906
received data! 52088666
received data! 52977521
received data! 47891181
received data! 46495700
received data! 47734233
received data! 51056363
received data! 49021306
received data! 46157373
received data! 50576143
received data! 53610073
received data! 50133563
received data! 50049387
received data! 50549154
calibration complete in 98 steps!
32


In [12]:
guessFrq = {}
for k, v in qpa._daqNode.pixelData.items():
    asic = k
    # print(asic)
    daqTicks = []
    asicTicks = []
    for daqTick, asicTick in v:
        daqTicks.append(daqTick)
        asicTicks.append(asicTick)
    guessFrq[asic] = (asicTicks[1] - asicTicks[0]) / (daqTicks[1] - daqTicks[0])  * qpa._daqNode.fOsc
for asic in qpa:
    key = f"({asic.row},{asic.col})"
    print(f"{key} percent diff: {(guessFrq[key] - asic.fOsc) / asic.fOsc * 100}")

(0,0) percent diff: 2.857177137948877e-07
(0,1) percent diff: 1.5931348898834817e-06
(0,2) percent diff: -8.623282041103211e-07
(0,3) percent diff: 1.0134098704071161e-06
(1,0) percent diff: -3.018106632468926e-07
(1,1) percent diff: 7.351877549335635e-08
(1,2) percent diff: -6.010605490052702e-07
(1,3) percent diff: 1.6881098404807808e-07
(2,0) percent diff: -1.688964437660719e-06
(2,1) percent diff: -4.4938804203868296e-07
(2,2) percent diff: 1.2490680713565108e-06
(2,3) percent diff: -2.593977173466662e-07
(3,0) percent diff: 6.492477850903155e-07
(3,1) percent diff: -4.420532074343671e-08
(3,2) percent diff: -2.8749087212095967e-07
(3,3) percent diff: 4.2354077518667335e-07
